In [2]:
!pip install portalocker

In [3]:
import numpy as np
import torch
from torch import nn
from torchtext.datasets import IMDB

from torchtext.data.functional import to_map_style_dataset
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from torch.utils.data import DataLoader
import re
from bs4 import BeautifulSoup

from tqdm import tqdm

In [4]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
file_name= '/kaggle/input/alice-wonderland-dataset/alice_in_wonderland.txt'
raw_text= open(file_name, 'r', encoding='utf-8').read()

In [6]:
train_dataset= IMDB(root="data", split=("train"))

In [7]:
train_dataset= to_map_style_dataset(train_dataset)

In [8]:
train_dataset= train_dataset[:100]

In [9]:
len(train_dataset)

100

In [10]:
def clean(text):
    text= text.lower()
    soup= BeautifulSoup(text)
    return soup.get_text()

In [11]:
raw_text= " ".join([clean(text) for lab, text in train_dataset]) 

In [12]:
len(raw_text)

132653

In [13]:
raw_text= raw_text.lower()
# raw_text= re.sub(r'\n', '', raw_text)
chars= sorted(list(set(raw_text)))
char_to_int= dict((c,i) for i,c in enumerate(chars))


In [14]:
n_chars= len(raw_text)
n_vocab= len(chars)
n_chars, n_vocab

(132653, 62)

In [15]:
seq_length= 100
dataX= []
dataY= []
for i in tqdm(range(0, n_chars - seq_length, 1)):
    seq_in= raw_text[i: i+seq_length]
    seq_out= raw_text[i+seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

100%|██████████| 132553/132553 [00:01<00:00, 85140.23it/s]


In [16]:
int_to_char= dict((i,c) for c,i in char_to_int.items())

In [17]:
n_patterns= len(dataX)
print("patterns", n_patterns)

patterns 132553


In [40]:
X= torch.tensor(dataX, dtype= torch.int).reshape(n_patterns, seq_length)
X= X
y= torch.tensor(dataY)
X.shape, y.shape

(torch.Size([132553, 100]), torch.Size([132553]))

In [21]:
# emb= nn.Embedding(n_vocab, 100)(torch.randint(0, n_vocab, (13, 100)))
# nn.LSTM(100, 256, num_layers=1, batch_first= True)(emb)

In [20]:
emb.shape

torch.Size([13, 100, 100])

In [41]:
class CharModel(nn.Module):
    def __init__(self):
        super(CharModel, self).__init__()
        self.embed= nn.Embedding(n_vocab, embedding_dim= 100)
        self.lstm= nn.LSTM(input_size=100, hidden_size=256, num_layers=3, batch_first=True)
        self.dropout= nn.Dropout(0.2)
        self.linear= nn.Linear(256, n_vocab)
        
    def forward(self, x):
        embeddings= self.embed(x)
        x, _= self.lstm(embeddings)
        x= self.dropout(x)
        logits= self.linear(x[:,-1])
        return logits

In [42]:
epochs= 100
batch_size= 128
model= CharModel().to(device)

In [43]:
train_dataset[10]

(1,
 'It was great to see some of my favorite stars of 30 years ago including John Ritter, Ben Gazarra and Audrey Hepburn. They looked quite wonderful. But that was it. They were not given any characters or good lines to work with. I neither understood or cared what the characters were doing.<br /><br />Some of the smaller female roles were fine, Patty Henson and Colleen Camp were quite competent and confident in their small sidekick parts. They showed some talent and it is sad they didn\'t go on to star in more and better films. Sadly, I didn\'t think Dorothy Stratten got a chance to act in this her only important film role.<br /><br />The film appears to have some fans, and I was very open-minded when I started watching it. I am a big Peter Bogdanovich fan and I enjoyed his last movie, "Cat\'s Meow" and all his early ones from "Targets" to "Nickleodeon". So, it really surprised me that I was barely able to keep awake watching this one.<br /><br />It is ironic that this movie is about

In [44]:
def guess():
#     start= np.random.randint(0, len(raw_text) - seq_length)
#     prompt= raw_text[start : start + seq_length]
    prompt= """It was great to see some of my favorite stars of 30 years ago including John Ritter, Ben Gazarra and"""
    prompt= prompt.lower()
    pattern= [char_to_int[char] for char in prompt]

    model.eval()
    print('Prompt: "%s"' % prompt)

    with torch.no_grad():
        for i in range(100):
            x = np.reshape(pattern, (1, len(pattern)))
            x= torch.tensor(x, dtype= torch.int) 
            x= x.to(device)
            prediction= model(x)
            index= int(prediction.argmax())
#             print(index)
        #     print(f"{int_to_char[prediction.argmax(1).item()]}")
        #     print(prediction)
            result= int_to_char[index]
    #         print(index)
            print(result, end="")
            pattern.append(index)
            pattern= pattern[1:]
    #     break
    print()
    print("Done")

In [45]:
guess()

Prompt: "it was great to see some of my favorite stars of 30 years ago including john ritter, ben gazarra and"
....................................................................................................
Done


In [46]:
#### optimizer= torch.optim.Adam(model.parameters())
loss_fn= nn.CrossEntropyLoss(reduction='sum')
optimizer= torch.optim.Adam(model.parameters(), lr= 1e-3)
loader= torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X, y), shuffle= True, batch_size= batch_size)


In [ ]:

best_model= None
best_loss= np.inf
loss= 0
for epoch in range(epochs):
    print(f"Epoch {epoch}")
    model.train()
    for X, y in tqdm(loader):
        X= X.to(device)
        y= y.to(device)
        y_pred= model(X)
#         print(y_pred.shape, y.shape)
        loss= loss_fn(y_pred, y)    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if loss<best_loss:
        best_model= model.state_dict()
        best_loss= loss
    print("Cross-entropy: %.4f" % (loss))
    guess()
    
#     model.eval()
    
#     with torch.no_grad():
#         for X, y in loader:
#             X= X.to(device)
#             y= y.to(device)
#             y_pred= model(X)
#             loss+= loss_fn(y_pred, y)
#         
        

Epoch 0


100%|██████████| 1036/1036 [00:46<00:00, 22.26it/s]


Cross-entropy: 146.5510
Prompt: "it was great to see some of my favorite stars of 30 years ago including john ritter, ben gazarra and"
 the the the the the the the the the the the the the the the the the the the the the the the the the
Done
Epoch 1


100%|██████████| 1036/1036 [00:45<00:00, 22.87it/s]


Cross-entropy: 141.7620
Prompt: "it was great to see some of my favorite stars of 30 years ago including john ritter, ben gazarra and"
 the story the story the story the story the story the story the story the story the story the story
Done
Epoch 2


...

100%|██████████| 1036/1036 [00:45<00:00, 22.94it/s]


Cross-entropy: 17.6968
Prompt: "it was great to see some of my favorite stars of 30 years ago including john ritter, ben gazarra and"
 audrey hepburn's last movies. i'll as one of his last movie, "cat's meow" and all his early ones fr
Done
Epoch 94


100%|██████████| 1036/1036 [00:45<00:00, 22.95it/s]


Cross-entropy: 22.5910
Prompt: "it was great to see some of my favorite stars of 30 years ago including john ritter, ben gazarra and"
 audrey hepburn's last movies. i enjoy the story of willie and went bankrupt where the gore is going
Done
Epoch 95


 96%|█████████▌| 997/1036 [00:43<00:01, 22.93it/s]

In [ ]:
model_path = "model_state.pth"

# Save the model state
torch.save(model.state_dict(), model_path)

In [ ]:
import pickle

with open('char_to_int.pkl', 'wb') as file:
    pickle.dump(char_to_int, file)

In [1]:
"hello"

'hello'